In [1]:
%cd /content/drive/MyDrive/senior-project/CNNs/

/content/drive/MyDrive/senior-project/CNNs


In [2]:
!pip install utils
!pip install livelossplot

In [3]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import utils
import os
%matplotlib inline
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Input, Dropout,Flatten, Conv2D
from tensorflow.keras.layers import BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model

from IPython.display import SVG, Image
from livelossplot import PlotLossesKeras
# from livelossplot import PlotLossesTensorFlowKeras
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print("Tensorflow version:", tf.__version__)

Found GPU at: /device:GPU:0
Tensorflow version: 2.8.0


In [4]:
data = pd.read_csv('fer2013.csv')

In [5]:
data

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training
...,...,...,...
35882,6,50 36 17 22 23 29 33 39 34 37 37 37 39 43 48 5...,PrivateTest
35883,3,178 174 172 173 181 188 191 194 196 199 200 20...,PrivateTest
35884,0,17 17 16 23 28 22 19 17 25 26 20 24 31 19 27 9...,PrivateTest
35885,3,30 28 28 29 31 30 42 68 79 81 77 67 67 71 63 6...,PrivateTest


In [6]:
for i in range(len(data)):
  data['pixels'][i] = data['pixels'][i].split(" ")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [7]:
for i in range(len(data)):
  data['pixels'][i] = [int(x) / 255 for x in data['pixels'][i]]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
data

,emotion,pixels,Usage
0,0,"[0.27450980392156865, 0.3137254901960784, 0.32...",Training
1,0,"[0.592156862745098, 0.5882352941176471, 0.5764...",Training
2,2,"[0.9058823529411765, 0.8313725490196079, 0.611...",Training
3,4,"[0.09411764705882353, 0.12549019607843137, 0.1...",Training
4,6,"[0.01568627450980392, 0.0, 0.0, 0.0, 0.0, 0.0,...",Training
...,...,...,...
35882,6,"[0.19607843137254902, 0.1411764705882353, 0.06...",PrivateTest
35883,3,"[0.6980392156862745, 0.6823529411764706, 0.674...",PrivateTest
35884,0,"[0.06666666666666667, 0.06666666666666667, 0.0...",PrivateTest
35885,3,"[0.11764705882352941, 0.10980392156862745, 0.1...",PrivateTest


In [10]:
for i in range(len(data)):
  data['pixels'][i] = " ".join(str(item) for item in data['pixels'][i])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
data["Usage"].value_counts()

Training       28709
PublicTest      3589
PrivateTest     3589
Name: Usage, dtype: int64

In [12]:
train_data = data[data.Usage == "Training"]

In [13]:
train_pixels = train_data.pixels.str.split(" ").tolist()
train_pixels = pd.DataFrame(train_pixels , dtype=int)
train_images = train_pixels.values
train_images = train_images.astype(np.float)

print(train_images)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3552: FutureWarning: Could not cast to int64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


[[0.2745098  0.31372549 0.32156863 ... 0.41568627 0.42745098 0.32156863]
 [0.59215686 0.58823529 0.57647059 ... 0.75686275 0.71764706 0.72156863]
 [0.90588235 0.83137255 0.61176471 ... 0.34509804 0.43137255 0.59607843]
 ...
 [0.29019608 0.31764706 0.34117647 ... 0.7372549  0.73333333 0.73333333]
 [0.87058824 0.89019608 0.79607843 ... 0.53333333 0.53333333 0.5254902 ]
 [0.76470588 0.78039216 0.80392157 ... 0.02352941 0.05882353 0.14901961]]


In [14]:
train_labels_flat = train_data["emotion"].values.ravel()
train_labels_count = np.unique(train_labels_flat).shape[0]
train_labels_count

7

In [15]:
def dense_to_one_hot(labels_dense, num_classes):
  num_labels = labels_dense.shape[0]
  
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

In [16]:
y_train = dense_to_one_hot(train_labels_flat, train_labels_count)
y_train = y_train.astype(np.uint8)
y_train.shape

(28709, 7)

In [17]:
test_data = data[data.Usage == "PublicTest"]
test_pixels = test_data.pixels.str.split(" ").tolist()

test_pixels = pd.DataFrame(test_pixels, dtype=int)
test_images = test_pixels.values
test_images = test_images.astype(np.float)

print(test_images.shape)

(3589, 2304)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  


In [18]:
test_labels_flat = test_data["emotion"].values.ravel()
test_labels_count = np.unique(test_labels_flat).shape[0]

y_test = dense_to_one_hot(test_labels_flat,test_labels_count)
y_test = y_test.astype(np.uint8)

print(y_test.shape)

(3589, 7)


In [19]:
x_train = train_images.reshape(-1, 48, 48, 1)
x_test = test_images.reshape(-1,48,48,1)

In [20]:
model = Sequential()

# Conv Block 1
model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 2
model.add(Conv2D(128,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(512,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 3
model.add(Conv2D(512,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# Conv Block 4
model.add(Conv2D(512,(5,5), padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))



model.add(Flatten())
# Fully connected Block 1
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# Fully connected Block 2
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

opt = Adam(lr=0.0005)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 48, 48, 64)        640       
                                                                 
 batch_normalization (BatchN  (None, 48, 48, 64)       256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 48, 48, 64)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 24, 24, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 24, 24, 64)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 128)       2

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [21]:
model.layers[0]

In [ ]:
epochs = 15


checkpoint = ModelCheckpoint("model_weights.h5",monitor='val_accuracy',
                            save_weights_only=True, mode='max',verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss' , factor=0.1, patience=2, min_lr=0.00001,model='auto')

callbacks = [PlotLossesKeras(), checkpoint, reduce_lr]

r = model.fit( x_train, y_train,  
                        epochs=30,
                        shuffle=True,
                        batch_size=200,
                        validation_data=(x_test, y_test),
                                              verbose=2)

Epoch 1/30
144/144 - 59s - loss: 1.8932 - accuracy: 0.2718 - val_loss: 1.8497 - val_accuracy: 0.2917 - 59s/epoch - 408ms/step
Epoch 2/30
144/144 - 37s - loss: 1.5834 - accuracy: 0.3956 - val_loss: 1.8311 - val_accuracy: 0.3316 - 37s/epoch - 257ms/step
Epoch 3/30
144/144 - 37s - loss: 1.4105 - accuracy: 0.4610 - val_loss: 1.7828 - val_accuracy: 0.3316 - 37s/epoch - 258ms/step
Epoch 4/30
144/144 - 37s - loss: 1.2999 - accuracy: 0.5037 - val_loss: 1.4303 - val_accuracy: 0.4494 - 37s/epoch - 258ms/step
Epoch 5/30
144/144 - 37s - loss: 1.2254 - accuracy: 0.5341 - val_loss: 1.3641 - val_accuracy: 0.4698 - 37s/epoch - 258ms/step
Epoch 6/30


In [ ]:
plt.plot(r.history['loss'],label = 'loss')
plt.plot(r.history['val_loss'],label ='val_loss')
plt.legend()

In [ ]:
# model_json = model.to_json()
# with open("model.json","w") as json_file:
#     json_file.write(model_json)